In [1]:
import re  # 정규표현식 사용                                     
from collections import Counter  # 단어 빈도 계산용                      

TEXT = "Hello! This is a test 문장입니다. NLP 전처리: 정규화/토큰화/인코딩 😊"  # 예시 텍스트      

def normalize(text: str) -> str:  # 정규화 함수(소문자화/특수문자 제거)      
    text = text.lower()  # 소문자화                                       
    text = re.sub(r"[^\w\s\.!?가-힣]", " ", text)  # 특수문자 제거(일부 문장부호는 유지) 
    text = re.sub(r"\s+", " ", text).strip()  # 공백 정리                    
    return text  # 정규화된 텍스트 반환                                    

def split_sentences(text: str) -> list[str]:  # 문장 분리 함수               
    parts = re.split(r"(?<=[\.\!\?])\s+", text)  # .!? 뒤에서 분리           
    parts = [s.strip() for s in parts if s.strip()]  # 빈 문장 제거          
    return parts  # 문장 리스트 반환                                        

def tokenize_word(sentence: str) -> list[str]:  # 워드 토큰화                
    return re.findall(r"[A-Za-z0-9_]+|[가-힣]+", sentence)  # 영문/숫자/한글 단어 추출

def tokenize_char(sentence: str) -> list[str]:  # 캐릭터 토큰화              
    return list(sentence)  # 글자 단위로 쪼갬                               

def tokenize_subword_simple(words: list[str]) -> list[str]:  # “간단” 서브워드 토큰화(데모) 
    pieces = []  # 결과 조각 리스트                                         
    for w in words:  # 각 단어에 대해                                      
        if len(w) <= 4:  # 짧으면 그대로 사용                                
            pieces.append(w)  # 그대로 추가                                 
        else:  # 길면 앞/뒤로 대충 분해(진짜 BPE/WordPiece 대체용 데모)       
            pieces.append(w[:3])  # 앞 3글자                                 
            pieces.append("##" + w[3:])  # 나머지(접두 표기 흉내)             
    return pieces  # 서브워드 조각 반환                                     

STOPWORDS = {  # 불용어(데모용, 필요에 맞게 확장)                            
    "is", "a", "the", "this", "to", "of", "and", "in", "on", "for",  # 영어 불용어 
    "은", "는", "이", "가", "을", "를", "에", "의", "와", "과", "도"  # 한글 조사 일부 
}  # 불용어 집합 끝                                                        

def remove_stopwords(tokens: list[str]) -> list[str]:  # 불용어 제거 함수     
    return [t for t in tokens if t not in STOPWORDS]  # 불용어 제외          

def stem_simple(token: str) -> str:  # 아주 단순한 어간 추출(영어 위주 데모)  
    t = token  # 작업용 복사                                                
    for suf in ("ing", "ed", "es", "s"):  # 흔한 접미사 목록                  
        if len(t) > 4 and t.endswith(suf):  # 너무 짧은 단어는 보호            
            t = t[: -len(suf)]  # 접미사 제거                                
            break  # 하나만 제거하고 종료                                   
    return t  # 스템 결과 반환                                              

LEMMA_MAP = {  # 표제어 추출(간단 매핑 데모: 실제론 사전/모델 필요)           
    "mice": "mouse",  # 예시 매핑                                          
    "children": "child",  # 예시 매핑                                      
    "went": "go",  # 예시 매핑                                              
}  # 매핑 끝                                                                

def lemmatize_simple(token: str) -> str:  # 아주 단순한 표제어 추출(데모)     
    return LEMMA_MAP.get(token, token)  # 매핑 있으면 치환, 없으면 그대로     

def build_vocab(token_sequences: list[list[str]], min_freq: int = 1) -> dict[str, int]:  # 어휘집합 생성
    counts = Counter(t for seq in token_sequences for t in seq)  # 전체 토큰 빈도 집계 
    vocab = {"[PAD]": 0, "[UNK]": 1}  # 특수 토큰(패딩/미등록) 기본 등록         
    for tok, c in counts.most_common():  # 빈도 높은 순으로 순회               
        if c >= min_freq and tok not in vocab:  # 최소 빈도 이상만              
            vocab[tok] = len(vocab)  # 새 ID 부여                            
    return vocab  # vocab(토큰→id) 반환                                     

def integer_encode(tokens: list[str], vocab: dict[str, int]) -> list[int]:  # 정수 인코딩
    return [vocab.get(t, vocab["[UNK]"]) for t in tokens]  # 모르면 UNK       

def pad_and_mask(ids: list[int], max_len: int, pad_id: int = 0) -> tuple[list[int], list[int]]:  # 패딩/마스킹
    ids = ids[:max_len]  # 길면 자르기                                      
    mask = [1] * len(ids)  # 실제 토큰은 1                                   
    while len(ids) < max_len:  # 짧으면 패딩 추가                             
        ids.append(pad_id)  # PAD 추가                                       
        mask.append(0)  # PAD 위치는 0                                       
    return ids, mask  # (패딩된 ids, 마스크) 반환                            

# --------- “통으로” 파이프라인 실행 예시 ---------  # 구분선                 
norm = normalize(TEXT)  # 1) 정규화                                         
sents = split_sentences(norm)  # 2) 문장 분리                               

word_tokens = [tokenize_word(s) for s in sents]  # 3) 워드 토큰화(문장별)      
word_tokens = [remove_stopwords(ws) for ws in word_tokens]  # 4) 불용어 제거(문장별)

stemmed = [[stem_simple(t) for t in ws] for ws in word_tokens]  # 5) 어간 추출(문장별)
lemmatized = [[lemmatize_simple(t) for t in ws] for ws in word_tokens]  # 6) 표제어(문장별)

subword_tokens = [tokenize_subword_simple(ws) for ws in word_tokens]  # 7) 서브워드 토큰화(데모)
char_tokens = [tokenize_char(s) for s in sents]  # 8) 캐릭터 토큰화(문장별)    

vocab = build_vocab(subword_tokens, min_freq=1)  # 9) 어휘집합(vocab) 생성(여기선 서브워드 기준)
encoded = [integer_encode(seq, vocab) for seq in subword_tokens]  # 10) 정수 인코딩(문장별)

max_len = 12  # 11) 패딩 기준 길이                                          
padded_and_masked = [pad_and_mask(seq, max_len, pad_id=vocab["[PAD]"]) for seq in encoded]  # 12) 패딩/마스킹

print("정규화:", norm)  # 결과 출력                                         
print("문장:", sents)  # 결과 출력                                          
print("워드:", word_tokens)  # 결과 출력                                    
print("어간:", stemmed)  # 결과 출력                                        
print("표제어:", lemmatized)  # 결과 출력                                   
print("서브워드(데모):", subword_tokens)  # 결과 출력                        
print("vocab size:", len(vocab))  # 결과 출력                                
print("인코딩:", encoded)  # 결과 출력                                      
print("패딩/마스크:", padded_and_masked)  # 결과 출력                        


정규화: hello! this is a test 문장입니다. nlp 전처리 정규화 토큰화 인코딩
문장: ['hello!', 'this is a test 문장입니다.', 'nlp 전처리 정규화 토큰화 인코딩']
워드: [['hello'], ['test', '문장입니다'], ['nlp', '전처리', '정규화', '토큰화', '인코딩']]
어간: [['hello'], ['test', '문장입니다'], ['nlp', '전처리', '정규화', '토큰화', '인코딩']]
표제어: [['hello'], ['test', '문장입니다'], ['nlp', '전처리', '정규화', '토큰화', '인코딩']]
서브워드(데모): [['hel', '##lo'], ['test', '문장입', '##니다'], ['nlp', '전처리', '정규화', '토큰화', '인코딩']]
vocab size: 12
인코딩: [[2, 3], [4, 5, 6], [7, 8, 9, 10, 11]]
패딩/마스크: [([2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ([4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ([7, 8, 9, 10, 11, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])]
